In [4]:
import pandas as pd

In [5]:
data_path = "../../../database"

df = pd.read_csv(f"{data_path}/silver/arquivo_anonimizado_v2.csv")


D_CURSO = pd.read_csv(f"{data_path}/gold/d_curso.csv")

D_PERIODO = pd.read_csv(f"{data_path}/gold/d_periodo.csv")

D_ALUNO = pd.read_csv(f"{data_path}/gold/d_aluno.csv")

In [6]:
def calcular_periodo_integralizado(periodo_atual, quant_integralizado):
    ano, semestre = map(int, periodo_atual.split('/'))
    quant_integralizado = quant_integralizado
    # Se quant_integralizado for ímpar, ajusta semestre e ano
    if quant_integralizado % 2 == 1:
        semestre += 1
        if semestre > 2:
            semestre = 1
            ano += 1
    
    # Calcula o novo ano considerando todos os semestres integralizados
    ano += quant_integralizado // 2

    return f"{ano}/{semestre}"

def obter_ultimo_periodo_cursado(row):
    disciplinas_str = row.get("disciplinas_cursadas", "")
    periodos_trancados = row.get("periodos_trancados", "")
    situacao_matricula = row.get("situacao_matricula_simplificada", "")
    cra_periodo = row.get("cra_periodo", "")
    cr_periodo = row.get("cr_periodo", "")
    periodo_ingresso_ufrj = row.get("periodo_ingresso_ufrj", "")

    # Caso disciplinas cursadas seja NaN ou vazio
    if pd.isna(disciplinas_str) or disciplinas_str == "":
        if pd.isna(cra_periodo) and pd.isna(cr_periodo):
            return periodo_ingresso_ufrj
        else:
            return cra_periodo.split(' - ')[0]

    # Extração dos períodos cursados
    disciplinas = disciplinas_str.split("\n")
    periodos = [disciplina.split(" - ")[0] for disciplina in disciplinas]

    # Encontrar o maior período cursado
    maior_periodo = max(periodos)

    # Considerar períodos trancados se a situação da matrícula for "Cancelada"
    if situacao_matricula == "Cancelada" and not pd.isna(periodos_trancados):
        trancados = periodos_trancados.replace(" ", "").split("|")
        maior_periodo_trancado = max(trancados)
        # Calcula o periodo seguinte ao trancado
        maior_periodo_trancado = calcular_periodo_integralizado(maior_periodo_trancado, 1)
        maior_periodo = max(maior_periodo, maior_periodo_trancado)

    return maior_periodo.replace("/0", "/1")

# Processar Trancamentos
def processar_trancamentos(row):
    if pd.isna(row):
        return row
    row = row.replace(' ', '')
    row = tuple(row.split('|'))
    return row

In [7]:
# Obtém o valor máximo da coluna 'periodoIngressoUFRJ'
periodo_atual = df['periodo_ingresso_ufrj'].max()

# Filtra os dados para obter apenas as linhas com 'situacao_matricula' igual a 'Ativa'
df_situacao_atual = df[(df['situacao_matricula_simplificada'] == "Ativa") | (df['situacao_matricula_simplificada'] == "Trancada")].copy()
df_cancel_concluido = df[(df['situacao_matricula_simplificada'] == "Cancelada") | (df['situacao_matricula_simplificada'] == "Concluido")].copy()

# Trancamento
df_trancada = df[~df['periodos_trancados'].isna()].copy()
df_trancada['situacao_matricula_simplificada'] = "Trancada"
df_trancada['periodo'] = df_trancada['periodos_trancados'].apply(processar_trancamentos)

In [8]:
df_situacao_atual['periodo'] = periodo_atual
df_trancada = df_trancada.explode('periodo')
df_cancel_concluido['periodo'] = df.apply(obter_ultimo_periodo_cursado, axis=1)

In [9]:
colunas_tb_fato = [
    "matricula_dre",
    "periodo",
    "cod_curso_ingresso",
    "curso_ingresso_ufrj",
    "cod_curso_atual",
    "curso_atual",
    "situacao_matricula",
    "situacao_matricula_simplificada"
]

In [10]:
df_ativa = df_situacao_atual[colunas_tb_fato]
df_trancada = df_trancada[colunas_tb_fato]
df_cancel_concluido = df_cancel_concluido[colunas_tb_fato]

# Usa a função concat para unir os DataFrames
df_final = pd.concat([df_ativa, df_trancada, df_cancel_concluido], axis=0)

# Reseta o índice se necessário
df_final.reset_index(drop=True, inplace=True)

In [11]:
# Mescla com a tabela D_PERIODO, selecionando apenas a coluna 'sk_d_periodo'
f_situacao_metricula = df_final.merge(
    D_PERIODO[["periodo", "sk_d_periodo"]], on="periodo", how="left"
)

# Mescla com a tabela D_CURSO e remove as colunas desnecessárias
f_situacao_metricula = f_situacao_metricula.merge(
    D_CURSO,
    on=["cod_curso_ingresso", "curso_ingresso_ufrj", "cod_curso_atual", "curso_atual"],
    how="left",
)

# Mescla com a tabela D_ALUNO e remove as colunas desnecessárias
f_situacao_metricula = f_situacao_metricula.merge(
    D_ALUNO,
    on="matricula_dre",
    how="left",
)

In [12]:
f_situacao_metricula = f_situacao_metricula[['sk_d_aluno', 'sk_d_periodo', 'sk_d_curso', 'situacao_matricula', 'situacao_matricula_simplificada']]

In [13]:
f_situacao_metricula.to_csv(f"{data_path}/gold/f_situacao_metricula.csv", index=False)